In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
from scipy.linalg import sqrtm, cholesky, cho_solve
import nonlinshrink as nls

def nlshrink_cov(Y, k=1):
    """
    Nonlinear shrinkage covariance estimation.
    
    Parameters:
    -----------
    Y : array-like, shape (n_samples, n_features)
        Data matrix
    k : int
        Number of factors (parameter for compatibility, may not be used)
    
    Returns:
    --------
    theta : array, shape (n_features, n_features)
        Shrinkage covariance estimate
    """
    # Using the non-linear-shrinkage package
    # This implements the Ledoit & Wolf nonlinear shrinkage method
    return nls.shrink_cov(Y)


def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    long_only : bool
        If True, falls back to GMV if MV produces negative weights
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        print('SINGULARITY')
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star


def compute_portfolio_metrics(returns, weights):
    """
    Compute portfolio return, variance, and Sharpe ratio.
    """
    portfolio_returns = returns @ weights
    mean_return = np.mean(portfolio_returns)
    variance = np.var(portfolio_returns, ddof=1)
    sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
    
    return {
        'return': mean_return,
        'variance': variance,
        'sharpe_ratio': sharpe_ratio
    }


def backtest_nodewise_gmv(df, 
                          test_start_date='2000-01-31', 
                          test_end_date='2003-12-31',
                          lookback_window=180,
                          transaction_cost=0.005,
                          verbose=True):
    """
    Backtest Nodewise + MSR strategy with monthly rebalancing,
    180-month rolling window, and NaN filtering as per the paper.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.005 = 50 bps)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # Track weights by permno (handles entry/exit)
    prev_weights_dict = {}
    prev_oos_returns_dict = {}
    prev_gross_return = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date)]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Assets: {p_current} with complete data")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), using prev weights")
            new_weights_dict = prev_weights_dict.copy()
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Nodewise Regression...")
                Sigma_hat = nlshrink_cov(Y_star)
                # Invert to get precision matrix
                try:
                    Theta_hat = np.linalg.inv(Sigma_hat)
                except np.linalg.LinAlgError:
                    # If inversion fails, use pseudo-inverse
                    if verbose:
                        print(f"  ⚠ Covariance matrix singular, using pseudo-inverse")
                    Theta_hat = np.linalg.pinv(Sigma_hat)

                # Compute expected returns (sample mean)
                mu = Y.mean(axis=0)
                
                if verbose:
                    print(f"  Computing MV weights...")
                w_star = mv_weights(Theta_hat, mu, target_return=0.01)
                
                # Create weights dictionary
                new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                new_weights_dict = prev_weights_dict.copy()

        # Normalize weights to sum to 1
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum, using previous weights")
            new_weights_dict = prev_weights_dict.copy()
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Get out-of-sample returns for current month
        oos_data = df[df['datadate'] == current_date]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        
        # Filter out NaN returns
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Find common assets between weights and returns
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, skipping period")
            continue
        
        # Filter to common assets and renormalize
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum after filtering, skipping period")
            continue
        
        # Compute gross portfolio return
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        
        # Sanity check
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ Invalid gross return: {gross_return}, skipping period")
            continue
        
        # Calculate transaction costs with proper weight adjustment
        if len(prev_weights_dict) > 0:
            # Step 1: Adjust ALL previous weights for returns
            adjusted_prev = {}
            
            for asset, prev_w in prev_weights_dict.items():
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    # Asset had weight but no return data
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
            
            # Step 2: Calculate turnover
            all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
            
            turnover = 0.0
            for asset in all_assets:
                old_w = adjusted_prev.get(asset, 0.0)
                new_w = common_weights.get(asset, 0.0)
                turnover += abs(new_w - old_w)
            
            # Transaction cost on end-of-period portfolio value
            tc = transaction_cost * (1 + gross_return) * turnover
        
        else:
            # First period: no previous positions, buying into everything
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * (1 + gross_return) * turnover
        
        # Net return
        net_return = gross_return - tc
        
        # Store results
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        if verbose:
            print(f"  Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    # Compute overall metrics
    if len(portfolio_returns) > 0:
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns, ddof=1)
        sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
        
        # Annualized metrics (monthly data)
        annual_return = mean_return * 12
        annual_volatility = np.sqrt(variance * 12)
        annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
        
        metrics = {
            'mean_return': mean_return,
            'variance': variance,
            'sharpe_ratio': sharpe_ratio,
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'annual_sharpe_ratio': annual_sharpe,
            'total_return': results_df['cumulative_return'].iloc[-1],
            'avg_turnover': np.mean(portfolio_turnover_list),
            'n_periods': len(portfolio_returns)
        }
    else:
        metrics = {
            'mean_return': 0,
            'variance': 0,
            'sharpe_ratio': 0,
            'annual_return': 0,
            'annual_volatility': 0,
            'annual_sharpe_ratio': 0,
            'total_return': 0,
            'avg_turnover': 0,
            'n_periods': 0
        }
    
    return results_df, metrics

In [2]:
# Assuming you have loaded your dataframe as 'df'
df = pd.read_csv('../../../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = (df.groupby('permno')['ret_excess'].shift(-1) )

In [3]:
results_df, metrics = backtest_nodewise_gmv(
    df,
    test_start_date='2015-01-31',  # Last date of training period
    test_end_date='2024-04-30',   # Last date of testing period
    lookback_window=180,
    transaction_cost=0.001
)
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")

STARTING BACKTEST

[1/112] Date: 2015-01-31
  Window: 2000-01-31 to 2014-12-31
  Assets: 242 with complete data
  Running Nodewise Regression...
  Computing MV weights...
  Gross:  0.03055 | Turnover: 2.8459 | TC: 0.002933 | Net:  0.02762

[2/112] Date: 2015-02-28
  Window: 2000-02-29 to 2015-01-31
  Assets: 242 with complete data
  Running Nodewise Regression...
  Computing MV weights...
  Gross: -0.01392 | Turnover: 0.6058 | TC: 0.000597 | Net: -0.01452

[3/112] Date: 2015-03-31
  Window: 2000-03-31 to 2015-02-28
  Assets: 240 with complete data
  Running Nodewise Regression...
  Computing MV weights...
  Gross:  0.00376 | Turnover: 0.7265 | TC: 0.000729 | Net:  0.00303

[4/112] Date: 2015-04-30
  Window: 2000-04-30 to 2015-03-31
  Assets: 242 with complete data
  Running Nodewise Regression...
  Computing MV weights...
  Gross:  0.01573 | Turnover: 0.4503 | TC: 0.000457 | Net:  0.01527

[5/112] Date: 2015-05-31
  Window: 2000-05-31 to 2015-04-30
  Assets: 242 with complete data
  Ru

In [4]:
metrics['mean_return']*12

0.07926046499334033

In [5]:
metrics['variance']*12

0.013563385099664236

In [6]:
metrics['avg_turnover']

0.5058946182932872